# 參數優化 - 將歷史報酬提升數倍的方式！

這個單元中，我們首先要來撰寫一個獲利的策略

寫一個策略，首先要來將我們之前爬到的數據拿出來

## 拿出歷史大盤數據

In [ ]:
from finlab.data import Data

data = Data()
twii = data.get('發行量加權股價指數')
twii = twii[(twii.index.second == 0)]['台股指數']

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")

%matplotlib inline

twii.plot()

In [ ]:
sma = twii.rolling(500).mean()
bias = twii / sma
ub =1+ bias.rolling(1000).std() * 2
lb = 1 - bias.rolling(1000).std() * 2

import matplotlib.pyplot as plt

plt.plot(bias['2008-03'].values)
plt.plot(ub['2008-03'].values)
plt.plot(lb['2008-03'].values)
plt.show()
plt.plot(twii['2008-03'].values)

## 乖離率策略

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

def strategy(m1=500, m2=1000, m3=1, m4=1, draw_plot=False):
    
    
    sma = twii.rolling(m1).mean()

    bias = (twii / sma)
    ub = 1 + bias.rolling(m2).std() * m3
    lb = 1 - bias.rolling(m2).std() * m4
    
    buy = (bias < lb) 
    sell = (bias > ub)

    hold = pd.Series(np.nan, index=sell.index)
    hold[buy] = 0
    hold[sell] = 1
    hold = hold.ffill()

    returns = twii.shift(-2) - twii.shift(-1)
    
    
    
    returns[hold == 0] = 0
    returns -= hold.diff().abs() * 3

    creturn = returns.cumsum()
    
    
    
    if draw_plot:
        (hold*1000).plot()
        creturn.plot()
    
     
    return creturn.dropna()[-1]

    
strategy(draw_plot=True)

## 乖離率優化

In [ ]:
vmax = 0
for m1 in range(500, 2000, 200):
    for m2 in range(400, 800, 20):
        v = strategy(m1, m2, 1, 1)
        if v > vmax:
            vmax = v
            print(vmax, m1, m2, 1, 1)

## 繪製出歷史報酬

In [ ]:
strategy(1300, 780, 1, 1, draw_plot=True)